In [140]:
using Clapeyron
using Plots
using CSV
using DataFrames
using LaTeXStrings

Define The Model For Pure NO2 and Set User Parameters

In [143]:
model = SAFTgammaMie([("nitrogen dioxide",["NO2"=>1])];userlocations=(Mw = [46.0055],
        epsilon = [318.75],
        sigma = [3.2004],
        lambda_a = [6],
        lambda_r = [17.989],
        vst = [2],
        S = [0.64494], 
        n_e=[1],
        n_H=[0],
        epsilon_assoc=Dict((("NO2","e"),("NO2","e")) => 5444.9),
        bondvol=Dict((("NO2","e"),("NO2","e")) => 0.10002e-30)));

Show the Model Parameters

In [146]:
@show model

    @show model.params.lambda_r.values
    @show model.params.lambda_a.values
    @show model.params.sigma.values
    @show model.params.epsilon.values
    @show model.params.segment.values
    @show model.params.shapefactor.values
    @show model.params.epsilon_assoc.values
    @show model.params.bondvol.values

model = SAFTgammaMie{BasicIdeal}("nitrogen dioxide")
model.params.lambda_r.values = [17.989;;]
model.params.lambda_a.values = [6.0;;]
model.params.sigma.values = [3.2004e-10;;]
model.params.epsilon.values = [318.75;;]
model.params.segment.values = [2]
model.params.shapefactor.values = [0.64494]
model.params.epsilon_assoc.values = Clapeyron.Compressed4DMatrix{Float64, Vector{Float64}}[5444.9]
model.params.bondvol.values = Clapeyron.Compressed4DMatrix{Float64, Vector{Float64}}[1.0002e-31]


Clapeyron.Compressed4DMatrix{Float64, Vector{Float64}} with 1 entry:
 (1, 1) >=< (1, 1): 1.0002e-31

Load In Experimental Critical Values From NIST

In [149]:
Tc_exp = 431.23;
Pc_exp = 101.0875;
Rhoc_exp = 12148.14;

#= units in kelvin, bar and mol/m3 =#

Calaculate Critical Values Using the Model

In [152]:
(Tc,Pc,Vc) = crit_pure(model);
Rhoc = 1/Vc;

@show Tc,Pc,Rhoc

#= units in kelvin, Pa and mol/m3 =#

(Tc, Pc, Rhoc) = (430.54961713256796, 9.54369986176589e6, 13593.66661727738)


(430.54961713256796, 9.54369986176589e6, 13593.66661727738)

Calculate Ratios

In [155]:
Tc_Ratio = Tc/Tc_exp;
Pc_Ratio = (Pc/1e5)/Pc_exp;
Rhoc_Ratio = Rhoc/Rhoc_exp;

@show Tc_Ratio,Pc_Ratio,Rhoc_Ratio

(Tc_Ratio, Pc_Ratio, Rhoc_Ratio) = (0.9984222274251976, 0.9441028674926069, 1.1189916001361015)


(0.9984222274251976, 0.9441028674926069, 1.1189916001361015)

Use temperatures from 300k below up to critical point spaced evenly with 120 points

In [158]:
N = 120;
T = LinRange(Tc-300,Tc,N);

Initialise Arrays

In [161]:
p = zeros(N);
Vl = zeros(N);
Vv = zeros(N);
Rho_l = zeros(N);
Rho_v = zeros(N);

Get the Model Results

In [164]:
for i in 1:N
    p[i],Vl[i],Vv[i] = saturation_pressure(model,T[i]);
    Rho_l[i] = 1/Vl[i];
    Rho_v[i] = 1/Vv[i];
end

Import The Experimental Data

In [167]:
Rho_l_exp = CSV.read("NO2_rho_liq_sat_Clap.csv",header=[1,2,3],DataFrame);
Rho_l_exp_HiT = CSV.read("NO2_rho_liq_sat_Hi-T_Clap.csv",header=[1,2,3],DataFrame);
Rho_v_exp = CSV.read("NO2_rho_vap_sat_Clap.csv",header=[1,2,3],DataFrame);
p_exp = CSV.read("NO2_vp_Clap.csv",header=[1,2,3],DataFrame);
p_exp_HiT = CSV.read("NO2_vp_Clap_Hi-T.csv",header=[1,2,3],DataFrame);

In [170]:
@show Rho_l_exp, Rho_l_exp_HiT, Rho_v_exp, p_exp, p_exp_HiT

(Rho_l_exp, Rho_l_exp_HiT, Rho_v_exp, p_exp, p_exp_HiT) = (24×2 DataFrame
 Row │ C_[method=saturation_rhol]_T  D_Column2_out_rho
     │ Float64                       Float64
─────┼─────────────────────────────────────────────────
   1 │                       273.15            32387.4
   2 │                       275.15            32300.5
   3 │                       277.15            32213.5
   4 │                       279.15            32126.6
   5 │                       281.15            32017.9
   6 │                       283.15            31931.0
   7 │                       286.15            31800.5
   8 │                       287.15            31735.3
   9 │                       288.15            31691.9
  10 │                       290.15            31604.9
  11 │                       291.15            31561.4
  12 │                       293.15            31452.8
  13 │                       294.15            31409.3
  14 │                       294.65            31387.6


(24×2 DataFrame
 Row │ C_[method=saturation_rhol]_T  D_Column2_out_rho 
     │ Float64                       Float64           
─────┼─────────────────────────────────────────────────
   1 │                       273.15            32387.4
   2 │                       275.15            32300.5
   3 │                       277.15            32213.5
   4 │                       279.15            32126.6
   5 │                       281.15            32017.9
   6 │                       283.15            31931.0
   7 │                       286.15            31800.5
   8 │                       287.15            31735.3
   9 │                       288.15            31691.9
  10 │                       290.15            31604.9
  11 │                       291.15            31561.4
  ⋮  │              ⋮                        ⋮
  15 │                       293.15            31452.8
  16 │                       273.15            32691.7
  17 │                       288.15            31691.9

Get Index Numbers

In [173]:
N_exp = size(p_exp)[1];
N_exp_high_T = size(p_exp_HiT)[1];

N_rho_exp = size(Rho_l_exp)[1];
N_rho_exp_high_T = size(Rho_l_exp_HiT)[1];
N_rho_exp_vap = size(Rho_v_exp)[1];

Make Vectors for this data

In [176]:
T_vp_exp = zeros(N_exp)
p_exp_T = zeros(N_exp)

T_vp_exp_high_T = zeros(N_exp_high_T);
p_exp_high_T = zeros(N_exp_high_T);

T_rho_exp = zeros(N_rho_exp);
rho_exp = zeros(N_rho_exp);

T_rho_exp_high_T = zeros(N_rho_exp_high_T);
rho_exp_high_T = zeros(N_rho_exp_high_T);

T_rho_vap = zeros(N_rho_exp_vap);
rho_exp_vap = zeros(N_rho_exp_vap);

Fill in the Vectors

In [179]:
T_vp_exp = p_exp[:,1];
p_exp_T = p_exp[:,2];

T_vp_exp_high_T = p_exp_HiT[:,1];
p_exp_high_T = p_exp_HiT[:,2];

T_rho_exp = Rho_l_exp[:,1];
rho_exp = Rho_l_exp[:,2];

T_rho_exp_high_T = Rho_l_exp_HiT[:,1];
rho_exp_high_T = Rho_l_exp_HiT[:,2];

T_rho_exp_vap = Rho_v_exp[:,1];
rho_exp_vap = Rho_v_exp[:,2];

Plot the graphs

In [182]:
plt = plot(xlims=(175,450),grid = :off, 
           framestyle = :box, 
           foreground_color_legend = nothing, 
           legend_font = font(12),
           legend = :topleft)

plot!(plt, T, p./1e5, 
      color = "black", 
      line = (:path, 3), 
      label = "Model")

plot!(plt, [Tc], [Pc]./1e5, 
      seriestype = :scatter, 
      color = "black", 
      markerstrokecolor = "black",
      marker = :square,
      line = (:scatter, 0.5), 
      label = "Model Critical Point")

plot!(plt, T_vp_exp, p_exp_T./1e5, 
      seriestype = :scatter, 
      color = "white", 
      markerstrokecolor = "black", 
      line = (:scatter, 0.4), 
      label = "Experimental Data")

plot!(plt, T_vp_exp_high_T, p_exp_high_T./1e5, 
      seriestype = :scatter, 
      color = "white", 
      markerstrokecolor = "black", 
      line = (:scatter, 0.4), 
      label = false)

plot!(plt, [431.15], [101.32], 
      seriestype = :scatter, 
      color = "white", 
      markerstrokecolor = "black",
      marker = :square,
      line = (:scatter, 0.4), 
      label = "Experimental Critical Point")

xlabel!(plt, "Temperature / (K)", xguidefontsize = 12)
ylabel!(plt, "Pressure / (bar)", yguidefontsize = 12)

savefig(plt, "NO2_Pressure_Temp")

"C:\\Users\\gk321\\OneDrive - Imperial College London\\Documents\\4th year\\Research Project\\NO2\\NO2_Pressure_Temp.png"

Plot the density curves

In [185]:
mw = (14+32)/1000;
colours = ["indianred2", "deepskyblue1"]
plt2 = plot(ylims=(200,:440),grid = :off, 
           framestyle = :box, 
           foreground_color_legend = nothing, 
           legend_font = font(12),
           legend = :bottomleft)

plot!(plt2, Rho_v.*mw, T, 
      color = colours[1], 
      line = (:path, 3), 
      label = "Vapour Density")

plot!(plt2, Rho_l.*mw, T,
      color = colours[2],
      line = (:path, 3),
      label = "Liquid Density")

plot!(plt2, [Rhoc].*mw, [Tc],
      seriestype=:scatter, 
      color="black", 
      markerstrokecolor="black", 
      marker = :square,
      line = (:scatter, 0.5),
      label = "Model Critical Point")

plot!(plt2, rho_exp.*mw, T_rho_exp,
      seriestype=:scatter,
      color="white",
      markerstrokecolor="black", 
      line = (:scatter, 0.5),
      label = "Experimental Data")

plot!(plt2, rho_exp_high_T.*mw, T_rho_exp_high_T,
      seriestype=:scatter,
      color="white",
      markerstrokecolor="black", 
      line = (:scatter, 0.5),
      label = false)

plot!(plt2, rho_exp_vap.*mw, T_rho_exp_vap,
      seriestype=:scatter,
      color="white",
      markerstrokecolor="black", 
      line = (:scatter, 0.5),
      label = false)

plot!(plt2, rho_exp_vap.*mw, T_rho_exp_vap,
      seriestype=:scatter,
      color="white",
      markerstrokecolor="black", 
      line = (:scatter, 0.5),
      label = false)

plot!(plt2, [Rhoc_exp*mw], [430.64],
      seriestype=:scatter,
      color="white",
      markerstrokecolor="black", 
      marker = :square,
      line = (:scatter, 0.5),
      label = "Experimental Critical Point")

xlabel!(plt2, "Density / (kg m⁻³)", yguidefontsize=12)
ylabel!(plt2, "Temperature / (K)", xguidefontsize=12)

savefig(plt2, "NO2_Density_Temp")

"C:\\Users\\gk321\\OneDrive - Imperial College London\\Documents\\4th year\\Research Project\\NO2\\NO2_Density_Temp.png"